In [6]:
# ❶ 라이브러리 로드 & 원본 JSON 읽기
import json, pathlib, pprint, textwrap

SRC = pathlib.Path("/Users/taeyoonkwack/Documents/HCLT-KACL-2025/Korean_Dialouge_Summarization/dataset/original/dev.json")
DST = pathlib.Path("/Users/taeyoonkwack/Documents/HCLT-KACL-2025/Korean_Dialouge_Summarization/dataset/finetuning_data/dev.json")

with SRC.open(encoding="utf-8") as f:
    raw = json.load(f)

In [7]:
def convo_to_text(conv):
    """리스트→‘{speaker}: 발화’ 줄바꿈 문자열"""
    lines = [f"{t['speaker']}: {t['utterance']}" for t in conv]
    return "\n".join(lines)

SYS_PROMPT = (
    "당신은 대화 내용을 듣고 요약하는 assistant입니다. "
    "아래의 대화 내용을 참고하고, 주어진 keyword에 맞게 요약문을 작성하세요."
)

In [8]:
processed = []
for samp in raw:
    cid   = samp["id"]
    convo = convo_to_text(samp["input"]["conversation"])
    kw    = ", ".join(samp["input"]["subject_keyword"])
    user_msg = textwrap.dedent(f"""\
        [대화]
        {convo}

        [keyword] 
        {kw}

        위의 대화 내용을 keyword에 맞게 요약문을 작성하시오. 
        요약문을 제외한 그 무엇도 출력하지 마시오. 
        Return your summarization only — no additional text.""")
    
    processed.append({
        "id": cid,
        "messages": [
            {"role": "system",    "content": SYS_PROMPT},
            {"role": "user",      "content": user_msg},
            {"role": "assistant", "content": samp["output"]}
        ]
    })

print("example =>"); pprint.pprint(processed[0], depth=3)


example =>
{'id': 'nikluge-2025-일상 대화 요약-dev-000001',
 'messages': [{'content': '당신은 대화 내용을 듣고 요약하는 assistant입니다. 아래의 대화 내용을 참고하고, '
                          '주어진 keyword에 맞게 요약문을 작성하세요.',
               'role': 'system'},
              {'content': '        [대화]\n'
                          '        SD2000691: 근데 인제 건강이 우리만 막 건강한다고 되는 게 아니잖아. '
                          '인제 애들도 건강도 챙겨줘야 되고 그러잖아. 그러니까 너는 애들한테 뭐 따로 먹이는 뭐 식품 '
                          '뭐 이런 거 있어?\n'
                          'SD2000692: 유산 유산균하고 비타민 그런데 애들이 잘 안 먹지. 나도 잘 안 먹는 데 '
                          '애들이 먹나? 근데 쫓아다니면서 챙겨줄 수도 없고 그런데 유산균은 꼭 먹이라 그러더라고. '
                          '유산균 장이 건강해야지 전체적으로 다 좋아진다고 그러더라고.\n'
                          'SD2000692: 그래서 될 수 있으면 비타민도 사놓기는 했는데 그거는 넘어가더라도 나는 '
                          '유산균은 꼭 먹으라고 얘기한다. 유산균이 어떤 사람은 공복에 먹으라 그러고 어떤 사람은 그냥 '
                          '아무 때나 먹으라고 하는데 모르겠어. 언제 먹는 게 좋은지\n'
                          'SD2000692: 그래서 name2이는 또 천식에 아토피까지 있잖아. 진짜 우리 '
       

In [9]:
with DST.open("w", encoding="utf-8") as f:
    json.dump(processed, f, ensure_ascii=False, indent=2)

print(f"✔️ Saved {len(processed):,} samples → {DST.resolve()}")

✔️ Saved 102 samples → /Users/taeyoonkwack/Documents/HCLT-KACL-2025/Korean_Dialouge_Summarization/dataset/finetuning_data/dev.json
